In [6]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
# csv 불러오기
Data_set = pd.read_csv("Datas.csv")

In [3]:
# X , y 나누기
X = Data_set.iloc[:,:-1]
y = Data_set.iloc[:,-1]
X.shape,y.shape

((207, 6), (207,))

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,
                                                y,
                                                test_size=0.3,
                                                # 랜덤스테이트로 섞어주기 
                                                random_state = 28
                                                )

In [47]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
import xgboost
from xgboost.sklearn import XGBClassifier
from lightgbm import LGBMClassifier
# 머신러닝 모델 다 써보기

In [57]:
# 스케일링해보기 -> 각도가 대부분 큼 
scaler= StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [58]:
# 결정나무
tree1 = DecisionTreeClassifier()
r = cross_val_score(tree1,X_train,y_train,cv=5)
r.mean()

0.8955665024630541

In [59]:
# 파라미터 줬을때
tree2 = DecisionTreeClassifier(max_depth=9)
result = cross_val_score(tree2,X_train,y_train,cv=5)
result.mean()

0.90935960591133

In [60]:
# knn
knn1 = KNeighborsClassifier()
r = cross_val_score(knn1,X_train,y_train,cv=5)
r.mean()

0.9445812807881773

In [61]:
# 파라미터 줬을때
knn2 = KNeighborsClassifier(n_neighbors=10)
r = cross_val_score(knn2,X_train,y_train,cv=5)
r.mean()

0.8610837438423644

In [64]:
# voting 해보기
voting_model = VotingClassifier(estimators=[('tree1',tree1), 
                                            ('tree2',tree2),
                                            ('knn1',knn1),
                                           ('knn2',knn2),
                                           ],
                               voting = 'soft')
r = cross_val_score(voting_model,X_train,y_train,cv=5)
r.mean()

0.9165024630541871

In [65]:
#렌덤포레스트  그리드 서치 해보기
params = {'n_estimators':[50,100,150,200],
         'max_depth':[3,5,7,9,10,12,15],
         'max_leaf_nodes':[2,4,6,8,10,12,14],
         'min_samples_split':[3,5,7,9,11,13]}
grid_model = RandomForestClassifier(random_state=100)
grid = GridSearchCV(grid_model,params,cv=5,scoring='accuracy',n_jobs=-1)
grid.fit(X_train,y_train)
print('최적의 하이퍼 파라미터 확인', grid.best_params_)
print('최고의 정확도 :',grid.best_score_)

최적의 하이퍼 파라미터 확인 {'max_depth': 5, 'max_leaf_nodes': 10, 'min_samples_split': 3, 'n_estimators': 50}
최고의 정확도 : 0.9302955665024631


In [66]:
best_ranfo = RandomForestClassifier(n_estimators=50,
                                   random_state=100,
                                   max_depth=5,
                                   max_leaf_nodes=10,
                                   min_samples_split=3)
r = cross_val_score(best_ranfo,X_train,y_train,cv=5)
r.mean()

0.9302955665024631

In [67]:
# ada 부스트 그리드 서치
params = {'n_estimators':[50,100,150,200],
         'learning_rate':[0.3,0.5,0.8,1]}
adb = AdaBoostClassifier(random_state= 100)
grid = GridSearchCV(adb,params,cv=5,scoring='accuracy',n_jobs=-1)
grid.fit(X_train,y_train)
print('최적의 하이퍼 파라미터 확인', grid.best_params_)
print('최고의 정확도 :',grid.best_score_)

최적의 하이퍼 파라미터 확인 {'learning_rate': 0.3, 'n_estimators': 50}
최고의 정확도 : 0.8677339901477833


In [68]:
best_adb = AdaBoostClassifier(random_state= 100,
                        learning_rate=0.3,
                        n_estimators=50)
r = cross_val_score(best_adb,X_train,y_train,cv=5)
r.mean()

0.8677339901477833

In [69]:
# GBoost
gbm = GradientBoostingClassifier(random_state=100)
grid = GridSearchCV(gbm,params,cv=5,scoring='accuracy',n_jobs=-1)
grid.fit(X_train,y_train)
print('최적의 하이퍼 파라미터 확인', grid.best_params_)
print('최고의 정확도 :',grid.best_score_)

최적의 하이퍼 파라미터 확인 {'learning_rate': 0.3, 'n_estimators': 50}
최고의 정확도 : 0.9096059113300493


In [70]:
best_gbm = GradientBoostingClassifier(random_state=100,
                                 learning_rate=0.3,
                                 n_estimators=150)
r = cross_val_score(best_gbm,X_train,y_train,cv=5)
r.mean()

0.9096059113300493

In [71]:
# XGBoost
params = {'n_estimators':[50,100,150,200],
         'max_depth':[3,5,7,9,10,12,15]}
xgb = XGBClassifier(random_state=150)
grid = GridSearchCV(xgb,params,cv=5,scoring='accuracy',n_jobs=-1)
grid.fit(X_train,y_train)
print('최적의 하이퍼 파라미터 확인', grid.best_params_)
print('최고의 정확도 :',grid.best_score_)

최적의 하이퍼 파라미터 확인 {'max_depth': 5, 'n_estimators': 200}
최고의 정확도 : 0.9096059113300493


In [72]:
best_xgb = XGBClassifier(random_state=100,
                    max_depth=5,
                    n_estimators=200)
r= cross_val_score(best_xgb,X_train,y_train,cv=5)
r.mean()

0.9096059113300493

In [73]:
# light gbm
lgb = LGBMClassifier()
params = {'n_estimators':[50,100,150,200],
         'max_depth':[3,5,7,9,10,12,15]}
grid = GridSearchCV(lgb,params,cv=5,scoring='accuracy',n_jobs=-1)
grid.fit(X_train,y_train)
print('최적의 하이퍼 파라미터 확인', grid.best_params_)
print('최고의 정확도 :',grid.best_score_)

최적의 하이퍼 파라미터 확인 {'max_depth': 5, 'n_estimators': 200}
최고의 정확도 : 0.9098522167487685


In [74]:
best_lgb = LGBMClassifier(max_depth=5,
n_estimators=200)
r= cross_val_score(best_lgb,X_train,y_train,cv=5)
r.mean()

0.9098522167487685

In [75]:
voting_model2 = VotingClassifier(estimators=[('best_adb',best_adb),
('best_xgb',best_xgb),
('best_gbm',best_gbm),
('best_lgb',best_lgb),
('best_ranfo',best_ranfo)])
r = cross_val_score(voting_model2,X_train,y_train,cv=5)
r.mean()

0.9096059113300493

In [77]:
# => scaling 한 knn이 가장 높다
# gridsearch로 best params 구하기 
knn = KNeighborsClassifier()
params = [{'n_neighbors': range(3,10)}]
grid = GridSearchCV(knn,params,cv=5,scoring='accuracy',n_jobs=-1)
grid.fit(X_train,y_train)
print('최적의 하이퍼 파라미터 확인', grid.best_params_)
print('최고의 정확도 :',grid.best_score_)

최적의 하이퍼 파라미터 확인 {'n_neighbors': 5}
최고의 정확도 : 0.9445812807881773


In [78]:
best_knn = KNeighborsClassifier(n_neighbors=5)
r = cross_val_score(best_knn,X_train,y_train,cv=5)
r.mean()

0.9445812807881773

In [81]:
# 저장
import pickle
knnPickle = open('knnpickle_file', 'wb') 
pickle.dump(best_knn, knnPickle)
knnPickle.close()

In [ ]:
# 나중에 모델 쓸때
# loaded_model = pickle.load(open('knnpickle_file', 'rb'))
# result = loaded_model.predict(X_test) 